<a href="https://colab.research.google.com/github/Muhammadsulton1/MIPT_Data_analys/blob/main/%D0%BF%D1%80%D0%BE%D0%B3%D0%BD%D0%BE%D0%B7_%D0%B2%D1%80%D0%B5%D0%BC%D1%8F%D0%BD%D0%BD%D1%8B%D1%85_%D1%80%D1%8F%D0%B4%D0%BE%D0%B2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd

from scipy import stats

import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')

import seaborn as sns

%matplotlib inline

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
ads = pd.read_csv('data/ads.csv', index_col=['Time'], parse_dates=['Time']).asfreq('H')
print(ads.shape)
ads.head()

In [ ]:
from sktime.utils.plotting import plot_series

plt.figure(figsize=(15, 6))
y = ads.Ads/10**3  # в тысячах :)

plot_series(y)
plt.title('Просмотры рекламы (тыс. часов)', fontsize=20, color='black')
plt.tick_params(axis = 'both', which = 'major', labelsize = 14);

2. Метрики качества прогноза
Рассмотрим основные и самые распространенные метрики качества прогнозов, которые, по большому счету, являются метриками для задачи регрессии и используются далеко не только во временных рядах.

Mean Absolute Error, интерпретируемая метрика, измеряется в тех же единицах, что и исходный ряд,


sklearn.metrics.mean_absolute_error

Median Absolute Error, также интерпретируемая метрика, однако её преимущество - нечувствительность (робастность) к выбросам в данных,

sklearn.metrics.median_absolute_error

Mean Squared Error, используется в большинстве случаев, сильнее наказывает модель за большие ошибки и меньше - за маленькие (парабола),


sklearn.metrics.mean_squared_error

Mean Squared Logarithmic Error, практически тоже самое, но значения предварительно логарифмируются, таким образом маленьким ошибкам также уделяется значительное внимание, обычно используется, если данным присущ экспоненциальный рост,


sklearn.metrics.mean_squared_log_error
Mean Absolute Percentage Error, как MAE, только в процентах, - удобно для объяснения заказчику качества прогноза,



def mean_absolute_percentage_error(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

Mean absolute scaled error, аналог
 для временных рядов. Тоже сравнивает прогноз по вашей модели с наивным и говорит насколько он оказался лучше.
Мы будем ниже пользоваться MAPE.

In [ ]:
def mape(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [ ]:
!pip install sktime

In [ ]:
import sktime
print(sktime.__version__)

Разбиение данных на обучение и контроль.

In [ ]:
y.size // 24

In [ ]:
from sktime.forecasting.model_selection import temporal_train_test_split

y_train, y_test = temporal_train_test_split(y, test_size=48)
plot_series(y_train, y_test, labels=["y_train", "y_test"])
print(y_train.shape[0], y_test.shape[0])

In [ ]:
from sktime.forecasting.base import ForecastingHorizon
fh = ForecastingHorizon(y_test.index, is_relative=False)
fh

4. Наивный прогноз
Можно построить наивный прогноз с помощью разных стратегий. Например, как последнее значение.

In [ ]:
from sktime.forecasting.naive import NaiveForecaster

forecaster = NaiveForecaster(strategy="last")
forecaster.fit(y_train)

y_pred = forecaster.predict(fh)

plot_series(y_train, y_test, y_pred, labels=["y_train", "y_test", "y_pred"]);

In [ ]:
mape(y_pred, y_test)

Либо среднего за последние window_length наблюдени

In [ ]:
forecaster = NaiveForecaster(strategy="mean", window_length=48)
forecaster.fit(y_train)

y_pred = forecaster.predict(fh)

plot_series(y_train, y_test, y_pred, labels=["y_train", "y_test", "y_pred"]);

In [ ]:
mape(y_pred, y_test)

Можно с помощью сезонного последнего значения.

In [ ]:
forecaster = NaiveForecaster(strategy="last", sp=24)
forecaster.fit(y_train)

y_pred = forecaster.predict(fh)

plot_series(y_train, y_test, y_pred, labels=["y_train", "y_test", "y_pred"]);

In [ ]:
mape(y_pred, y_test)

Самый лучший из наивных прогнозов, в виде последнего сезонного значения, мы и будем использовать в качестве базового решения. Обратите внимание, что если бы модель содержала тренд, было бы логично как-то учесть его в вашем наивном прогнозе. Иначе любая модель будет его превосходить по качеству, что не очень честно.

5. ETS - модели
ETS(ANN)

In [ ]:
from sktime.forecasting.exp_smoothing import ExponentialSmoothing

# ETS(ANN)   старое название: Simple exponential smoothing
forecaster = ExponentialSmoothing()
forecaster.fit(y_train)

y_pred = forecaster.predict(fh)

plot_series(y_train, y_test, y_pred, labels=["y_train", "y_test", "y_pred"]);

In [ ]:
mape(y_test, y_pred)

In [ ]:
forecaster.get_fitted_params()

Если попросить интервальные прогнозы, пакет скажет, что они не реализованы. Вы знаете как их сделать и можете попробовать реализовать для этого пакета на базе statsmodels. Как строить интервальные прогнозы в нём, мы посмотрим ниже.

In [ ]:
forecaster.predict(fh, return_pred_int=True, alpha=0.95)

ETS(AAA)

In [ ]:
# ETS(ANN)   старое название: Simple exponential smoothing
forecaster = ExponentialSmoothing(trend="add", seasonal="add", sp=24)
forecaster.fit(y_train)

y_pred = forecaster.predict(fh)

plot_series(y_train, y_test, y_pred, labels=["y_train", "y_test", "y_pred"]);

In [ ]:
mape(y_test, y_pred)

In [ ]:
forecaster.get_fitted_params()

Auto-ETS
Можно перебрать все ETS модели и выбрать лучшую по какому-то критерию.

In [ ]:
from sktime.forecasting.ets import AutoETS

# Делаем перебор и выбираем лучшую модель по информационному критерию
forecaster = AutoETS(auto=True, n_jobs=-1, information_criterion='bic')
forecaster.fit(y_train)

y_pred = forecaster.predict(fh)
plot_series(y_train, y_test, y_pred, labels=["y_train", "y_test", "y_pred"]);

In [ ]:
mape(y_test, y_pred)

Доверительные интервалы
К сожалению, в рамках sktime доверительные интервалы для ETS-моделей ещё не реализованы. Вы можете помочь развитию пакета и реализовать их самостоятельно! :)

Сейчас ETS-модель опирается в этом пакете на модуль exponential_smoothing из пакета statsmodels. В нём доверительные интервалы не предусмотрены. В этом же пакете есть модуль statespace. Там доверительные интервалы уже есть.

In [ ]:
import statsmodels.api as sm

ets_aaa = sm.tsa.statespace.ExponentialSmoothing(y_train, trend=True, seasonal=24)
ets_aaa_res = ets_aaa.fit()
print(ets_aaa_res.summary())

In [ ]:
h = 48 # test_size
df_forecast = ets_aaa_res.get_forecast(h).summary_frame()
df_forecast.index = fh
df_forecast.head()

In [ ]:
fig, ax = plot_series(y_train, y_test, df_forecast['mean'], labels=["y_train", "y_test", "y_pred"])
ax.fill_between(
    ax.get_lines()[-1].get_xdata(),
    df_forecast["mean_ci_lower"],
    df_forecast["mean_ci_upper"],
    alpha=0.2,
    color=ax.get_lines()[-1].get_c(),
    label=f"95% prediction intervals",
)
ax.legend(loc='lower left');

In [ ]:
mape(y_test, df_forecast['mean'])

Диагностика модели
Чтобы доверительный интервалы были корректными, надо чтобы выполнялись предпосылки модели. Ошибки должны быть независимо нормально распределены с нулевым средним и одинаковой дисперсией. Давайте посмотрим на них

In [ ]:
ets_aaa_res.plot_diagnostics(figsize=(10, 8));

Левый верхний график показывает динамику остатков во времени вокруг среднего. Мы видим, что математическое ожидание действительно совпадает с нулём. С дисперсией также всё в норме.

Правый нижний график называется коррелелограмма. По данным считается автокорреляция
. Голубым нарисован доверительный интервал для этих корреляций. Если корреляция лежит внутри интервала, гипотеза о её равенстве нулю не отвергается. Судя по всему
 и
 ненулевые. Это может быть связано с тем, что мы некорректно задали параметры, которые отвечают за сезонность.

На второстепенной диагонали находится распределение остатков и диаграмма квантиль-квантиль. Хвосты распределения оказываются тяжёлыми.

Выполнения предпосылок модели можно попытаться добиться, поперебирав гиперпараметры модели. Либо можно попробовать сделать остатки более похожими на нормальные, сделав для исходного ряда преобразование Бокса-Кокса.

Если нас не интересуют интервальные прогнозы, а интересуют только точечные, можно не добавиваться выполнения предпосылки о нормальности. Также можно заменить нормальное распределение ошибок на какое-то другое и заняться максимизацией своей собственной функции правдоподобия. Правда придётся реализовать её вручную по аналогии с тем, что мы делали на неделе про правдоподобие.

6. Кросс-валидация
Выше мы оценивали модель на тренировочном периоде, а дальше строили прогноз на весь тестовый. Дальше мы считали ошибку. Это не очень корректно, так как строить прогноз на неделю вперёд сложнее, чем на сутки вперёд.

Гораздо корректнее сделать кросс-валидацию со скользящим окном и сравнить модели между собой на разных горизонтах прогнозирования.

In [ ]:
from sklearn.model_selection import TimeSeriesSplit

Посмотрим в документацию. Там пишут, что размер тренировочных выборок на -ой итерации:
i * n_samples // (n_splits + 1) + n_samples % (n_splits + 1)

Размер тестовой выборки на -ой итерации:

n_samples//(n_splits + 1)

In [ ]:
z = np.arange(10)

tscv = TimeSeriesSplit(n_splits=3) #, max_train_size=5)

for train_index, test_index in tscv.split(z):
    print("TRAIN:", train_index, "TEST:", test_index)

Настроим разбиение так, чтобы в тесте всегда было 7 наблюдений.

In [ ]:
n_samples = y.size
n_splits = y.size // 8

# начинаем строить модель с 27 наблюдения
n_samples // (n_splits + 1) + n_samples % (n_splits + 1)

In [ ]:
# в тесте каждый раз по 7 наблюдений
n_samples//(n_splits + 1)

In [ ]:
from collections import defaultdict

n_splits = y.size // 8          # для того, чтобы в тесте каждый раз было 7 наблюдений
h = n_samples//(n_splits + 1)   # число наблюдений в тестовой выборке

M = defaultdict(list) # для метрик

def mape(y_true, y_pred):
    return np.abs((y_true - y_pred) / y_true)* 100

tscv = TimeSeriesSplit(n_splits=n_splits)
for train, test in tscv.split(y):
    y_tr, y_ts = y[train].values, y[test].values

    # Первая модель
    model = sm.tsa.statespace.ExponentialSmoothing(y_tr, trend=True, seasonal=24).fit()
    y_hat = model.get_forecast(h).summary_frame()['mean']
    M['ETS(A A A)'].append(mape(y_ts, y_hat.values))

    # Вторая модель
    model = sm.tsa.statespace.ExponentialSmoothing(y_tr, trend=True, seasonal=24, damped_trend=True).fit()
    y_hat = model.get_forecast(h).summary_frame()['mean']
    M['ETS(A Ad A)'].append(mape(y_ts, y_hat.values))

In [ ]:
# Ошибки при кросс-валидации из текущей точки на 7 дней вперёд
np.array(M['ETS(A A A)'])

In [ ]:
plt.figure(figsize=(12,6))
for m in M.keys():
    plt.plot(np.arange(1,h+1),np.array(M[m]).mean(axis=0), label=m)

plt.xlabel('Горизонт прогнозирования', fontsize=16)
plt.ylabel('MAPE', fontsize=16)
plt.legend(fontsize=16);

Видно, что на  дня модели прогнозируют примерно одинаково, а дальше качество расходится.

7. LOO кросс-валидация
LOO кросс-валидацию можно сделать более элегантно, с помощью оконных функций в pandas.

In [ ]:
def timeseries_cv(y, metric=mape, treshhold=0.8):

    # последний день всегда в тестовой выборке
    y_train = y[:-1].values
    y_test = y[-1:].values

    # на каждой выборке обучаем модель
    model = sm.tsa.statespace.ExponentialSmoothing(y_train, trend=True, seasonal=24)
    res = model.fit()

    # делаем прогноз и находим качество прогноза
    y_hat = res.get_forecast(1).summary_frame()['mean']
    return metric(y_test, y_hat)

Функция прогнозирует на день вперёд и вычисляет ошибку.

In [ ]:
timeseries_cv(y)

In [ ]:
min_period = int(0.8*y.size)

In [ ]:
# расширяющееся окно
quality = y.expanding(min_period).apply(lambda x: timeseries_cv(x))
quality.dropna().mean()

In [ ]:
# скользящее окно
quality = y.rolling(min_period).apply(lambda x: timeseries_cv(x))
quality.dropna().mean()